In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata
from datetime import datetime, timedelta

LEAGUE_ID = 1519305834
SEASON = 2026

# ---------------------------------------------------------------------
# Name normalization (remove accents)
# ---------------------------------------------------------------------
def normalize_name(name: str) -> str:
    if not isinstance(name, str):
        return ""
    s = unicodedata.normalize("NFKD", name)
    return "".join(c for c in s if not unicodedata.combining(c)).replace("*","").strip()

# ---------------------------------------------------------------------
# ESPN Team ID → NBA Team abbr mapping
# (Static ESPN → BR mapping for schedule lookups)
# ---------------------------------------------------------------------
ESPN_TEAM_MAP = {
    1:"ATL", 2:"BOS", 3:"NOP", 4:"CHI", 5:"CLE", 6:"DAL", 7:"DEN",
    8:"DET", 9:"GSW", 10:"HOU", 11:"IND", 12:"LAC", 13:"LAL",
    14:"MIA", 15:"MIL", 16:"MIN", 17:"BKN", 18:"NYK", 19:"ORL",
    20:"PHI", 21:"PHO", 22:"POR", 23:"SAC", 24:"SAS", 25:"OKC",
    26:"UTA", 27:"WAS", 28:"TOR", 29:"MEM", 30:"CHA"
}

# ---------------------------------------------------------------------
# Fetch ESPN rosters with player info
# ---------------------------------------------------------------------
def fetch_rosters(league_id, season):
    base = "https://lm-api-reads.fantasy.espn.com/apis/v3/games/fba"
    url = f"{base}/seasons/{season}/segments/0/leagues/{league_id}"

    resp = requests.get(url, params={"view":["mTeam","mRoster"]})
    resp.raise_for_status()
    data = resp.json()

    rosters = {}

    for team in data["teams"]:
        raw_name = team.get("name") or f"{team.get('location','')} {team.get('nickname','')}"
        team_name = normalize_name(raw_name)

        players = []
        for entry in team["roster"]["entries"]:
            p = entry["playerPoolEntry"]["player"]
            players.append({
                "playerId": p["id"],
                "fullName": normalize_name(p["fullName"]),
                "proTeamId": p.get("proTeamId"),
                "injuryStatus": p.get("injuryStatus","ACTIVE"),
            })

        rosters[team_name] = players

    return rosters

# ---------------------------------------------------------------------
# Fetch per-game stats from Basketball Reference
# (Must include FGM/FGA/FTM/FTA)
# ---------------------------------------------------------------------
def fetch_per_game_stats(url):
    
    tables = pd.read_html(url)
    df = tables[0]

    df = df[df["Rk"] != "Rk"].dropna(subset=["Player"])

    df["Player"] = df["Player"].apply(normalize_name)
    df.loc[df["Player"] == "Jimmy Butler", "Player"] = "Jimmy Butler III"

    return df

# ---------------------------------------------------------------------
# Get number of games this week per NBA team
# Example: start_date="2025-11-17", end_date="2025-11-23"
# ---------------------------------------------------------------------
import pandas as pd
from datetime import datetime
import unicodedata

# All NBA teams (BR format → ESPN format handled later)
NBA_TEAMS = {
    "Atlanta Hawks": "ATL",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BRK",
    "Charlotte Hornets": "CHO",
    "Chicago Bulls": "CHI",
    "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL",
    "Denver Nuggets": "DEN",
    "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW",
    "Houston Rockets": "HOU",
    "Indiana Pacers": "IND",
    "Los Angeles Clippers": "LAC",
    "Los Angeles Lakers": "LAL",
    "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP",
    "New York Knicks": "NYK",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Philadelphia 76ers": "PHI",
    "Phoenix Suns": "PHO",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR",
    "Utah Jazz": "UTA",
    "Washington Wizards": "WAS"
}

def normalize_team_name(name: str):
    """Fix accents, strip *, normalize whitespace."""
    if not isinstance(name, str):
        return None
    name = unicodedata.normalize("NFKD", name)
    name = "".join(c for c in name if not unicodedata.combining(c))
    name = name.replace("*", "").strip()
    return name

def fetch_games_this_week(start_date: datetime, end_date: datetime):
    """
    Returns complete weekly team schedule:
    e.g.: { "LAL": 3, "BOS": 4, ... } — including zeroes for teams with no games.
    """

    # Basketball-Reference has separate pages:
    # NBA_2026_games-october.html
    # NBA_2026_games-november.html
    # ...
    months = [
        "october", "november", "december", "january",
        "february", "march", "april"
    ]

    all_games = []

    # Fetch all months
    for month in months:
        url = f"https://www.basketball-reference.com/leagues/NBA_2026_games-{month}.html"
        try:
            tables = pd.read_html(url)
        except:
            continue  # Month may not exist yet

        df = tables[0]

        # Remove header repeat rows & invalid rows
        df = df[df["Date"] != "Date"]
        df = df.dropna(subset=["Date", "Visitor/Neutral", "Home/Neutral"])

        # Parse date
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
        df = df.dropna(subset=["Date"])

        all_games.append(df)

    if not all_games:
        return {}

    df = pd.concat(all_games, ignore_index=True)

    # Filter week
    mask = (df["Date"] >= start_date) & (df["Date"] <= end_date)
    weekly = df[mask]

    # Initialize all teams to 0
    counts = {abbr: 0 for abbr in NBA_TEAMS.values()}

    # Count games
    for _, row in weekly.iterrows():
        away = normalize_team_name(row["Visitor/Neutral"])
        home = normalize_team_name(row["Home/Neutral"])

        # Skip invalid rows
        if away not in NBA_TEAMS or home not in NBA_TEAMS:
            continue

        counts[NBA_TEAMS[away]] += 1
        counts[NBA_TEAMS[home]] += 1

    return counts


# ---------------------------------------------------------------------
# Project stats based on games this week & correct FG%/FT% calculation
# ---------------------------------------------------------------------
def project_weekly_team_stats(rosters, stats_df, weekly_games, categories):
    rows = []

    for team_name, players in rosters.items():

        totals = {c:0 for c in categories}
        totals["FGM"] = 0
        totals["FGA"] = 0
        totals["FTM"] = 0
        totals["FTA"] = 0

        for p in players:

            name = p["fullName"]
            nba_team = ESPN_TEAM_MAP.get(p["proTeamId"])
            if nba_team is None:
                continue

            games = weekly_games.get(nba_team, 0)
            if games == 0:
                continue

            row = stats_df[stats_df["Player"].str.lower() == name.lower()]
            if row.empty:
                print(f"Not found: {name}")
                continue

            r = row.iloc[0]

            # Add FGM/FGA/FTM/FTA
            try:
                totals["FGM"] += float(r["FG"]) * games
                totals["FGA"] += float(r["FGA"]) * games
                totals["FTM"] += float(r["FT"]) * games
                totals["FTA"] += float(r["FTA"]) * games
            except:
                pass

            for c in categories:
                if c in ["FG%", "FT%"]:
                    continue
                if c in r:
                    try:
                        totals[c] += float(r[c]) * games
                    except:
                        pass

        # Compute percentages now
        totals["FG%"] = totals["FGM"] / totals["FGA"] if totals["FGA"] > 0 else 0
        totals["FT%"] = totals["FTM"] / totals["FTA"] if totals["FTA"] > 0 else 0

        rows.append({"team":team_name, **totals})

    return pd.DataFrame(rows)

# ---------------------------------------------------------------------
# Full Run
# ---------------------------------------------------------------------
def main():
    stats_url = "https://www.basketball-reference.com/leagues/NBA_2026_per_game.html"

    rosters = fetch_rosters(LEAGUE_ID, SEASON)
    stats_df = fetch_per_game_stats(stats_url)

    # Define the week (example)
    week_start = datetime(2025, 12, 1)
    week_end   = datetime(2025, 12, 7)

    weekly_games = fetch_games_this_week(week_start, week_end)

    categories = ["PTS","TRB","AST","STL","BLK","3P","TOV","FG%","FT%"]

    df = project_weekly_team_stats(rosters, stats_df, weekly_games, categories)

    print("\nProjected Weekly Stats:\n")
    print(df)
    return df

if __name__ == "__main__":
    df=main()
